# Data cleaner + Variable Searcher
### Add to the beginning of your notebook
**Imports the data and does the recommended cleaning. Adds 3 functions modified from variable searcher to return detailed information about variables.**
Variable names:
- cols: the list of variables
- data: the original dataframe

Functions:
- get_matching_keywords: takes a list of strings and returns all variables who contain at least one keyword
- get_var_details: takes a variable name and returns a list of questions corresponding to which dataset the variable is from
- get_column_info: takes a variable name and returns a list of every question associated with the variable as well as information about the type and amount of responses to the question.

In [2]:
import pandas as pd
cols = pd.read_csv("var_names.csv")
data = pd.read_csv('CSCS_data_anon.csv', low_memory=False,
                        na_values=["9999", "", " ", "Presented but no response", "NA"])
empty = (data.isna().sum()==data.shape[0])
data = data[empty.index[~empty]] # keep non empty columns only
data = data[data.REMOVE_case=='No'].copy()

In [ ]:
def get_matching_keywords(keywords: list[str], group_restriction: list[str]=None, year_restriction: list[int]=None, strict: bool=False) -> list[str]:
    """
    Returns the names of the columns whose string contains any of the keywords
    Can restrict to specific years by passing a list of years to year restriction
    Same for group restriction
    Strict means that a variable must be found in all years and all groups listed, non strict means it only needs to satisfy one year or one group
    """
    global data
    assert group_restriction == None or all({res.lower() in [None, "cohort", "cross"] for res in group_restriction})
    assert year_restriction == None or all({str(res) in [None, "2021", "2022", "2023"] for res in year_restriction})
    matching = [col for col in data.columns if any(keyword.lower() in col.lower() for keyword in keywords)]
    if group_restriction == None and year_restriction == None: return matching
    group_restriction = [res.lower() for res in group_restriction]
    output = []
    for col in matching:
        details = get_var_details(col)
        if not details: continue
        if strict:
            if all({any(str(year) in d for q, d in details) for year in year_restriction}) and \
                all({any(group in d for q, d in details) for group in group_restriction}):
                output.append(col)

        else:
            if any(any(str(year) in d for q, d in details) for year in year_restriction) and \
                any(any(group in d for q, d in details) for group in group_restriction):
                output.append(col)

    return output

def get_matching_keywords_question(keywords: list[str], group_restriction: list[str]=None, year_restriction: list[int]=None, strict: bool=False) -> list[str]:
    """
    Returns the names of the columns whose question contains any of the keywords
    Can restrict to specific years by passing a list of years to year restriction
    Same for group restriction
    Strict means that a variable must be found in all years and all groups listed, non strict means it only needs to satisfy one year or one group
    """
    global data
    assert group_restriction == None or all({res.lower() in [None, "cohort", "cross"] for res in group_restriction})
    assert year_restriction == None or all({str(res) in [None, "2021", "2022", "2023"] for res in year_restriction})
    matching = [col for col in data.columns if any(keyword.lower() in "".join([str(thing[0]) for thing in get_var_details(col)]).lower() for keyword in keywords)]
    if group_restriction == None and year_restriction == None: return matching
    group_restriction = [res.lower() for res in group_restriction]
    output = []
    for col in matching:
        details = get_var_details(col)
        if not details: continue
        if strict:
            if all({any(str(year) in d for q, d in details) for year in year_restriction}) and \
                all({any(group in d for q, d in details) for group in group_restriction}):
                output.append(col)

        else:
            if any(any(str(year) in d for q, d in details) for year in year_restriction) and \
                any(any(group in d for q, d in details) for group in group_restriction):
                output.append(col)

    return output

def get_from_dataset(datasets: list[str], strict: bool=False) -> list[str]:
    """
    Get the variables who appear in the list of given datasets
    If strict, the variable must appear in all of the listed datasets
    """
    global data
    output = []
    for col in data.columns:
        details = get_var_details(col)
        if not details: continue
        if all(dd in [ds for q, ds in details] for dd in datasets): output.append(col)

    return output


def get_var_details(var_name: str) -> list[tuple[str, str]]:
    """
    Returns a list of pairs detailing the question and datasets 
    corresponding to the given variable name
    """
    global cols
    filtered_data = cols[cols['new_var'] == var_name]
    
    info = []
    result: pd.DataFrame = filtered_data[['new_var', 'text', 'dataset']]
    for index, row in result.iterrows():
        info.append((row['text'], row['dataset']))

    return info

def get_column_info(column_name: str) -> None:
    """
    Returns information about a given column
    """
    global data, cols
    column_info: pd.DataFrame = data[column_name]
    print(f"{' Information about: ' + column_name + ' ':-^70}")
    details = get_var_details(column_name)
    if not details: print(f"{' No associated question found ':-^70}")
    for question, dataset in details:
        print(f"{' Dataset: ' + dataset + ' ':-^70}")
        print(f"{' Question: ' + question + ' ':-^70}")
        print(f"{'':-^70}\n")
    print(f"Number of NA: {column_info.isna().sum()}")
    print(column_info.value_counts())
    print("")

def get_datasets(column_name: str) -> list[str]:
    """
    Returns a list of datasets a variable name is found in
    """
    datasets = []
    for question, dataset in get_var_details(column_name):
        datasets.append(dataset)
    return datasets

In [13]:
# Example usage
words = ["WORK_friend"]
to_examine = get_matching_keywords(words)
for col in to_examine:
    get_column_info(col)

------- Information about: VOLUNTEERISM_volunteer_work_friends -------
------------------------ Dataset: 2022_cross -------------------------
 Question: Which of the following is true about your volunteer work? (Check all that apply) - I volunteered with friends.  
----------------------------------------------------------------------

Number of NA: 8978
VOLUNTEERISM_volunteer_work_friends
Not Selected                   579
I volunteered with friends.    461
Name: count, dtype: int64

 Information about: CONNECTION_lubben_social_network_friends_subscale_see 
------------------------ Dataset: 2023_cross -------------------------
 Question: Please answer each of the following questions about the people you know: - How many of your friends do you see or hear from at least once a month? 
----------------------------------------------------------------------

Number of NA: 8143
CONNECTION_lubben_social_network_friends_subscale_see
2             428
1             338
3             298
4     

Interests:
- CONNECTION_activities_meeting_work_p3m 
- CONNECTION_preferred_frequency_hang_out 
- CONNECTION_activities_visited_friends_p3m 
- CONNECTION_activities_new_friend_p3m / CONNECTION_activities_new_friend_last - DO busy people make new friends?
- CONNECTION_social_days_friends_p7d_grouped / CONNECTION_social_days_friends_p7d 
- CONNECTION_social_time_friends_p7d_grouped / CONNECTION_social_time_friends_p7d 
- CONNECTION_preference_time_friends_grouped / CONNECTION_preference_time_friends 
- CONNECTION_meaningful_time_friends_grouped
- CONNECTION_social_num_close_friends 
- CONNECTION_lubben_social_network_friends_subscale_see 

Money?
- CONNECTION_close_friends_same_more_money 
- CONNECTION_close_friends_same_less_money 
- CONNECTION_close_friends_same_money 
- FRIENDSHIP_hall_friendship_standards_self_money 

Humor
- FRIENDSHIP_hall_friendship_standards_self_sense_of_humor 

Work friends
- WORK_friends_num 

In [14]:
to_examine = ["CONNECTION_activities_meeting_work_p3m", "CONNECTION_preferred_frequency_hang_out", "CONNECTION_activities_visited_friends_p3m", "CONNECTION_activities_new_friend_p3m", "CONNECTION_activities_new_friend_last", "CONNECTION_social_days_friends_p7d_grouped", "CONNECTION_social_days_friends_p7d", "CONNECTION_social_time_friends_p7d_grouped", "CONNECTION_social_time_friends_p7d", "CONNECTION_preference_time_friends_grouped", "CONNECTION_preference_time_friends", "CONNECTION_meaningful_time_friends_groupe", "CONNECTION_social_num_close_friends", "CONNECTION_lubben_social_network_friends_subscale_see"]
to_examine = get_matching_keywords(to_examine)
for col in to_examine:
    get_column_info(col)

---- Information about: CONNECTION_activities_visited_friends_p3m ----
------------------------ Dataset: 2021_cross -------------------------
 Question: In the PAST THREE MONTH, how often have you... - visited with FRIENDS at your / their home? 
----------------------------------------------------------------------

------------------------ Dataset: 2022_cohort ------------------------
 Question: In the PAST THREE MONTH, how often have you... - visited with FRIENDS at your / their home? 
----------------------------------------------------------------------

------------------------ Dataset: 2023_cohort ------------------------
 Question: In the PAST THREE MONTH, how often have you... - visited with FRIENDS at your / their home? 
----------------------------------------------------------------------

------------------------ Dataset: 2023_cross -------------------------
 Question: In the PAST THREE MONTH, how often have you... - visited with FRIENDS at your or their home? 
------------